BASED ON Week 1 Day 3 LAB Exercise

This program evaluates different LLM outputs who are acting as customer service representative and are replying to an irritated customer.
OpenAI 40 mini, Gemini, Deepseek, Groq and Ollama are customer service representatives who respond to the email and OpenAI 3o mini analyzes all the responses and ranks their output based on different parameters.

In [1]:
# Start with imports -
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, display

In [ ]:
# Always remember to do this!
load_dotenv(override=True)

In [ ]:
# Print the key prefixes to help with any debugging

openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

In [4]:
persona = "You are a customer support representative for a subscription bases software product."
email_content = '''Subject: Totally unacceptable experience

Hi,

I’ve already written to you twice about this, and still no response. I was charged again this month even after canceling my subscription. This is the third time this has happened.

Honestly, I’m losing patience. If I don’t get a clear explanation and refund within 24 hours, I’m going to report this on social media and leave negative reviews.

You’ve seriously messed up here. Fix this now.

– Jordan

'''

In [5]:
messages = [{"role":"system", "content": persona}]

In [ ]:
request = f"""A frustrated customer has written in about being repeatedly charged after canceling and threatened to escalate on social media.
Write a calm, empathetic, and professional response that Acknowledges their frustration, Apologizes sincerely,Explains the next steps to resolve the issue
Attempts to de-escalate the situation. Keep the tone respectful and proactive. Do not make excuses or blame the customer."""
request += f" Here is the email : {email_content}]"
messages.append({"role": "user", "content": request})
print(messages)

In [ ]:
messages

In [8]:
competitors = []
answers = []
messages = [{"role": "user", "content": request}]

In [ ]:
# The API we know well
openai = OpenAI()
model_name = "gpt-4o-mini"

response = openai.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
model_name = "gemini-2.0-flash"

response = gemini.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
deepseek = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com/v1")
model_name = "deepseek-chat"

response = deepseek.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
model_name = "llama-3.3-70b-versatile"

response = groq.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)


In [ ]:
!ollama pull llama3.2

In [ ]:
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
model_name = "llama3.2"

response = ollama.chat.completions.create(model=model_name, messages=messages)
answer = response.choices[0].message.content

display(Markdown(answer))
competitors.append(model_name)
answers.append(answer)

In [ ]:
# So where are we?

print(competitors)
print(answers)


In [ ]:
# It's nice to know how to use "zip"
for competitor, answer in zip(competitors, answers):
    print(f"Competitor: {competitor}\n\n{answer}")


In [16]:
# Let's bring this together - note the use of "enumerate"

together = ""
for index, answer in enumerate(answers):
    together += f"# Response from competitor {index+1}\n\n"
    together += answer + "\n\n"

In [ ]:
print(together)

In [18]:
judge = f"""You are judging the performance of {len(competitors)} who are customer service representatives in a SaaS based subscription model company.
Each has responded to below grievnace email from the customer:

{request}

Evaluate the following customer support reply based on these criteria. Assign a score from 1 (very poor) to 5 (excellent) for each:

1. Empathy:
Does the message acknowledge the customer’s frustration appropriately and sincerely?

2. De-escalation:
Does the response effectively calm the customer and reduce the likelihood of social media escalation?

3. Clarity:
Is the explanation of next steps clear and specific (e.g., refund process, timeline)?

4. Professional Tone:
Is the message respectful, calm, and free from defensiveness or blame?

Provide a one-sentence explanation for each score and a final overall rating with justification.

Here are the responses from each competitor:

{together}

Do not include markdown formatting or code blocks. Also create a table with 3 columnds at the end containing rank, name and one line reason for the rank"""


In [ ]:
print(judge)

In [20]:
judge_messages = [{"role": "user", "content": judge}]


In [ ]:
# Judgement time!

openai = OpenAI()
response = openai.chat.completions.create(
    model="o3-mini",
    messages=judge_messages,
)
results = response.choices[0].message.content
print(results)


In [ ]:
print(results)